# 卷积神经网络

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim


In [2]:
# 简单得卷积
x = torch.rand(1, 1, 28, 28)
# 第一个参数是输入1个channel, 因为是黑白的
# 第二个参数是多少个kernel, 也就是3个kernel,输出3个feature
# 3x3 然后步长1, padding是0
layer = nn.Conv2d(1, 3, kernel_size=3, stride=1, padding=0)
output = layer.forward(x)
print(output.shape)

layer = nn.Conv2d(1, 3, kernel_size=3, stride=1, padding=1)
output = layer.forward(x)
print(output.shape)

layer = nn.Conv2d(1, 3, kernel_size=3, stride=2, padding=1)
output = layer.forward(x)
print(output.shape)


torch.Size([1, 3, 26, 26])
torch.Size([1, 3, 28, 28])
torch.Size([1, 3, 14, 14])


[W NNPACK.cpp:64] Could not initialize NNPACK! Reason: Unsupported hardware.


In [3]:
# 注意, 卷积核是可以被训练的, 它同时会带有一个bias也就是偏执参数b (常量)
print(layer.weight)  # 打印kernel, 这里3个channel, 1个input, 3x3的kernel, 因此是[3,1,3,3]
print(layer.weight.shape)
print(layer.bias.shape)


Parameter containing:
tensor([[[[-0.0687, -0.2829, -0.1221],
          [ 0.3210, -0.1693,  0.1862],
          [-0.0617, -0.1443,  0.1890]]],


        [[[ 0.1810, -0.0718,  0.2155],
          [ 0.0242, -0.1810, -0.1930],
          [ 0.0338, -0.1208, -0.1184]]],


        [[[ 0.1495, -0.2583,  0.1091],
          [-0.0657, -0.0613,  0.1949],
          [-0.2283,  0.0678,  0.1106]]]], requires_grad=True)
torch.Size([3, 1, 3, 3])
torch.Size([3])


In [4]:
# 也可以使用函数式接口进行卷积
x = torch.randn(1, 3, 28, 28)  # 注意输入输出的channel数必须相同
w = torch.rand(16, 3, 5, 5)
b = torch.rand(16)
out = F.conv2d(x, w, b, stride=1, padding=1)
print(out.shape)

out = F.conv2d(x, w, b, stride=1, padding=2)
print(out.shape)

out = F.conv2d(x, w, b, stride=2, padding=2)
print(out.shape)


torch.Size([1, 16, 26, 26])
torch.Size([1, 16, 28, 28])
torch.Size([1, 16, 14, 14])


## Pooling

上下采样

In [5]:
# 下采样, maxpooling
x = out

layer = nn.MaxPool2d(2, stride=2)
out = layer(x)
print(out.shape)

# 也可以用 nn.AvgPool2d
out = F.avg_pool2d(x, 2, stride=2)
print(out.shape)


torch.Size([1, 16, 7, 7])
torch.Size([1, 16, 7, 7])


In [6]:
# 上采样, 放大2倍, 将原始数据直接放大. 最后两个维度翻倍
x = out
out = F.interpolate(x, scale_factor=2, mode="nearest")
print(out.shape)

out = F.interpolate(x, scale_factor=3, mode="nearest")
print(out.shape)


torch.Size([1, 16, 14, 14])
torch.Size([1, 16, 21, 21])


## Relu

In [8]:
layer = nn.ReLU(inplace=True)
out = layer(x)
print(out.shape)
out = F.relu(x)
print(out.shape)


torch.Size([1, 16, 7, 7])
torch.Size([1, 16, 7, 7])


## Batch Normalization

当数据进行梯度求解的时候用的. 权重进入sigmoid参数的时候很可能是杂乱无章的, 因此很可能出现梯度离散的情况. 所以可以使用normalize进行过变换. 换为一个$N(0,\sigma)^2$的范围.

如果输入数据的范围差距可能很大, 比如$x_1$的范围在0.1-0.2之间,$x_2$的范围在1-255之间, 因此权重的梯度形状就非常的杂乱无章, 因此我们可以进行normalize, 让其输入都控制在一个范围内. 然后梯度的求解尽量快并且不会出现杂乱的问题. 

![20](./assets/20.png)

实现比较简单, 我们对每一个通道进行normalize

```python
normalize = transforms.Normalize(mean=[0.485,0.456,0.406],std=[0.229,0.224,0.225])
```

实现就是batch normalization, 也就是根据batch进行normalize

比如我现在有一个batch, batch中存在6个图片3种颜色[6,3,784]. 我们对其进行normalization之后, 我们会得到3个均值和3个方差.

![20](./assets/20.png)

然后我们就可以将其缩放到$N(0,1)$的正态分布, 然后我们在对这个正态分布增加一个偏执参数$\beta$

In [9]:
# 线性数据进行操作
x = torch.rand(100, 16, 784)
layer = nn.BatchNorm1d(16)
out = layer(x)
print(out.shape)

print(layer.running_mean)
print(layer.running_var)


torch.Size([100, 16, 784])
tensor([0.0499, 0.0500, 0.0500, 0.0500, 0.0500, 0.0500, 0.0501, 0.0500, 0.0500,
        0.0502, 0.0500, 0.0499, 0.0500, 0.0500, 0.0499, 0.0501])
tensor([0.9083, 0.9083, 0.9084, 0.9084, 0.9083, 0.9083, 0.9084, 0.9083, 0.9083,
        0.9083, 0.9083, 0.9083, 0.9083, 0.9083, 0.9083, 0.9083])


In [10]:
# 2d数据进行操作
x = torch.rand(1, 16, 7, 7)
layer = nn.BatchNorm2d(16)
out = layer(x)
print(out.shape)

# 对于2d来说是存在可导的系数,和编制参数
print(layer.weight)  # 对应的就是gamma, 也就是权重缩放的参数
print(layer.bias)  # 对应的就是beta偏执参数

# 全局数据集
print(layer.running_mean)
print(layer.running_var)


torch.Size([1, 16, 7, 7])
Parameter containing:
tensor([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
       requires_grad=True)
Parameter containing:
tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       requires_grad=True)
tensor([0.0533, 0.0475, 0.0476, 0.0502, 0.0506, 0.0537, 0.0464, 0.0548, 0.0479,
        0.0496, 0.0481, 0.0470, 0.0521, 0.0531, 0.0545, 0.0497])
tensor([0.9103, 0.9075, 0.9084, 0.9066, 0.9079, 0.9095, 0.9083, 0.9087, 0.9080,
        0.9098, 0.9095, 0.9098, 0.9094, 0.9076, 0.9102, 0.9087])


In [11]:
# 在test的时候需要执行eval, 这样才能使用全局的均值和方差
print(layer.eval())
out = layer(x)
print(out.shape)


BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
torch.Size([1, 16, 7, 7])


## Resnet

定义残差网络一般使用的是残差网络的block, 我们将整个block进行训练, 或者进行bypass. 

在定义一层以后, 我们定义一个`extra`层, 这个`extra`层就是输入层, 正常的卷积层和`extra`的结果相加. 然后在反向传播的时候, 我们同时训练这个`extra`层. 如果这一层表现得好, 我们的输出层kernel的属性就会非常小, 体现的训练层就会非常清晰, 反之, 我们的`extra`层的kernel就会非常大, 就会跳过这一层的重要性. 大概是这样.

`extra`存在的本质就是input和output的shape调整到一样的大小, 使用1x1的kernel进行卷积处理

In [ ]:
# 残差网络的基本单元
class ResBlock(nn.Module):
    def __init__(self, ch_in, ch_out) -> None:
        """
        shape必须一致, 一般我们的channel不一致, 一般使用padding和相同kernel大小的话, size是一致的.
        输入和输出不一定相同, 需要注意的是我们不太关心输入的大小,因为使用卷积核可以得到我们所需要的featuremap大小
        假设我们在这里使用了ch_in = 64, ch_out = 256
        """
        # 输出ch_out个kernel产生的featuremap, 也就是[256,H,W]
        self.conv1 = nn.Conv2d(ch_in, ch_out, kernel_size=3, stride=1, padding=1)
        self.bn1 = nn.BatchNorm2d(ch_out)
        # 输出ch_out个kernel产生的featuremap, 也就是[256,H,W]
        self.conv2 = nn.Conv2d(ch_out, ch_out, kernel_size=3, stride=1, padding=1)
        self.bn2 = nn.BatchNorm2d(ch_out)
        # 到此处我们得到了一个ch_out也就是[256,H,W]的图片

        self.extra = nn.Sequential()
        if ch_out != ch_in:
            # [b, ch_in, h, w] => [b, ch_out, h, w]
            self.extra = nn.Sequential(
                nn.Conv2d(ch_in, ch_out, kernel_size=1, stride=1),
                nn.BatchNorm2d(ch_out),
            )

    def forward(self, x):
        """
        我们在前向传播中定义了relu层, 而不是在网络中定义的
        """
        out = F.relu(self.bn1(self.conv1(x)))
        out = self.bn2(self.conv2(out))
        # 对应位置数值进行相加, 就是shortcut和卷积结果相加
        out = self.extra(x) + out
        return out


## nn.Model 类

所有的层需要继承这个类来进行前向传播. 他提供了现成的模块

- Linear
- Relu
- sigmoid
- conv2d
- convtranspose2d
- DropOut

容器: Sequential. 可以将所有的层来进行容器堆叠.
```python
self.net = nn.Sequential(
    nn.Conv2d(1,32,5,1,1)
    nn.MaxPool2d(2,2)
    nn.ReLU(True),
    nn.BatchNormal2d(32),

    nn.Conv2d(32,64,3,1,1)
    nn.ReLU(True),
    nn.BatchNormal2d(64),

    nn.Conv2d(64,64,3,1,1)
    nn.MaxPool2d(2,2)
    nn.ReLU(True),
    nn.BatchNormal2d(64),
    ...
)
```

![22](./assets/22.png)

同时Sequencial可以进行各种意义上的嵌套, 只要是一层一层的结构就可以被嵌套. 因此可以定义自己想要的类. 如果需要的话就可以查看children来观察所有的children. 可以直接将整个网络结构搬入gpu中

```python
device = torch.device('cuda')
net = Net()
net.to(device)
```

In [ ]:
class MyLinear(nn.Module):
    def __init__(self, inp, outp):
        super(MyLinear, self).__init__()

        # requires_grad = True
        self.w = nn.Parameter(torch.randn(outp, inp))
        self.b = nn.Parameter(torch.randn(outp))

    def forward(self, x):
        x = x @ self.w.t() + self.b
        return x


## 应用相当广泛
class Flatten(nn.Module):
    def __init__(self):
        super(Flatten, self).__init__()

    def forward(self, input):
        return input.view(input.size(0), -1)


class TestNet(nn.Module):
    def __init__(self):
        super(TestNet, self).__init__()

        self.net = nn.Sequential(
            nn.Conv2d(1, 16, stride=1, padding=1),
            nn.MaxPool2d(2, 2),
            Flatten(),
            nn.Linear(1 * 14 * 14, 10),
        )

    def forward(self, x):
        return self.net(x)


class BasicNet(nn.Module):
    def __init__(self):
        super(BasicNet, self).__init__()

        self.net = nn.Linear(4, 3)

    def forward(self, x):
        return self.net(x)


class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()

        self.net = nn.Sequential(BasicNet(), nn.ReLU(), nn.Linear(3, 2))

    def forward(self, x):
        return self.net(x)


保存模型结构: 

```python
net.load_state_dict(torch.load('ckpt.mdl'))
torch.save(net.state_dict(), 'ckpt.mdl')
```

## 数据增强

略

```python
train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=True, download=True,
                   transform=transforms.Compose([
                       transforms.RandomHorizontalFlip(),
                       transforms.RandomVerticalFlip(),
                       transforms.RandomRotation(15),
                       transforms.RandomRotation([90, 180, 270]),
                       transforms.Resize([32, 32]),
                       transforms.RandomCrop([28, 28]),
                       transforms.ToTensor(),
                       # transforms.Normalize((0.1307,), (0.3081,))
                   ])),
    batch_size=batch_size, shuffle=True)
```